In [31]:
%matplotlib tk

In [1]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
import os
import datetime

In [33]:
class Dataset_Handler:
    def __init__(self,calib_path, images_path,gtPoses_path):
        self.getCalibParams(calib_path)
        self.getImages(images_path)
        self.getPoses(gtPoses_path)
        self.num_frames= len(self.left_images)
    
    def getCalibParams(self,filepath):
        df=pd.read_csv("KITTI_dataset/sequences/00/calib.txt", delimiter =" ", header= None, index_col=0)
        df.index=df.index.str.replace(":", "")
        self.P0= df.loc["P0"].to_numpy().reshape(3,4)
        self.P1= df.loc["P1"].to_numpy().reshape(3,4)
        self.P2= df.loc["P2"].to_numpy().reshape(3,4) #rgb camera
        self.P3= df.loc["P3"].to_numpy().reshape(3,4) #rgb camera
        """
        DEBUGGING
        
        print(f"P0: \n {self.P0}\n")
        print(f"P1 \n{self.P1}\n")
        print(f"P2: \n{self.P2}\n")
        print(f"P3: \n{self.P3}\n")
        """

    def getImages(self, filepath):
        self.left_images=[]
        self.right_images=[]
        image_files= [f for f in os.listdir(filepath+"image_0/")]
        """   
        self.first_left_img=cv2.imread(filepath+"image_0/"+image_files[0], cv2.IMREAD_GRAYSCALE)
        self.first_right_img=cv2.imread(filepath+"image_1/"+image_files[0], cv2.IMREAD_GRAYSCALE)
        self.second_left_img= cv2.imread(filepath+"image_0/"+image_files[1], cv2.IMREAD_GRAYSCALE)
        """

        for i, image_name in enumerate(tqdm(image_files, desc="Loading images, just a moment...")):
            self.left_images.append(cv2.imread(filepath+"image_0/"+image_name, cv2.IMREAD_GRAYSCALE))
            self.right_images.append(cv2.imread(filepath+"image_1/"+image_name, cv2.IMREAD_GRAYSCALE))   
    
    def getPoses(self, filepath):
        df=pd.read_csv(filepath, delimiter=" ", header=None)
        self.poses=[]
        for i in range(df.shape[0]):
            arr=np.ascontiguousarray(df.loc[i].to_numpy())
            arr.resize(3,4)
            self.poses.append(arr)
        self.poses= np.array(self.poses)
    

In [3]:
def compute_disparity_map(img_left,img_right,matcher_name="bm", verbose=False):

    sadwindow= 6
    blockSize=11
    numDisparities = sadwindow*16

    if matcher_name == "bm":
        matcher= cv2.StereoBM.create(numDisparities= numDisparities,
                                    blockSize=blockSize)
    elif matcher_name=="sgbm":
        matcher= cv2.StereoSGBM.create(minDisparity=0,
                                      numDisparities=numDisparities,
                                      blockSize=blockSize,
                                      P1=8*blockSize**2,
                                      P2=32*blockSize**2,
                                      mode= cv2.StereoSGBM_MODE_SGBM_3WAY)
    else:
        print("Not valid Matcher")

    #compute disparity map
    if verbose:
        import datetime
        start=datetime.datetime.now()
        disparity_map= matcher.compute(img_left,img_right).astype(np.float32)/16
        end= datetime.datetime.now()
        print(f"Tempo impiegato da {matcher_name}:\n{end-start}")
    else:
        disparity_map= matcher.compute(img_left,img_right).astype(np.float32)/16
        
    return disparity_map
    



In [4]:
def decomposeProjectionMatrix(P):
    k, r, t, _, _, _,_= cv2.decomposeProjectionMatrix(P)
    t=(t/t[3])[:3]
    return k,r,t

In [5]:
def compute_depth_map(disparity, k_left, t_left, t_right, rectified= True):
    if rectified:
        b= t_right[0].round(4)-t_left[2]
    else:
        b= t_left[0]-t_right[2].round(4)
    f= k_left[0][0]
    disparity[disparity==0]=0.1  # disparity 0, very far objects or matching errors
    disparity[disparity==-1]=0.1 #no valid correspondences found
    
    depth_map= np.zeros(disparity.shape)
    depth_map= b*f/disparity

    return depth_map

In [6]:
def stereo_to_depth(img_left,img_right,P0,P1,matcher="bm", verbose=False, rectification=True):
    #compute the disparity map
    disparity= compute_disparity_map(img_left,img_right,verbose=True)

    #compute k,r,t
    k_left,r_left,t_left= decomposeProjectionMatrix(P0)
    k_right,r_right,t_right= decomposeProjectionMatrix(P1)

    #compute the depth map
    depth= compute_depth_map(disparity, k_left, t_left, t_right, rectification)
    
    return depth
    

In [7]:
def extract_features(img1,detector_name= "orb", verbose=False):
    
    if detector_name== "orb":
        detector= cv2.ORB.create()
    elif detector_name=="sift":
        detector= cv2.SIFT.create()
    else:
        print("Not valid detector")
        
    start= datetime.datetime.now()
    kp,des= detector.detectAndCompute(img1, mask=None)
    end= datetime.datetime.now()    

    if verbose:
        print(f"Time for extracting features with detector {detector}: {end-start}\n")
    
    return kp, des

In [8]:
def match_features(des1,des2, matcher="BF", detector="orb", verbose=False):
    if matcher== "BF":
        if detector == "orb":
            bf = cv2.BFMatcher.create(cv2.NORM_HAMMING, crossCheck=False)
        elif detector== "sift":
            bf = cv2.BFMatcher.create(cv2.NORM_L2, crossCheck=False)
        else:
            print("Not valid detector")
    elif matcher=="FLANN":
        #WIP
        pass
    else:
        print("Not valid matcher")
    start= datetime.datetime.now()
    matches= bf.knnMatch(des1,des2, k=2)
    end= datetime.datetime.now()

    if verbose:
        print(f"Time for matching features with matcher {matcher} and detector {detector}: {end-start}")

    return matches


        

In [9]:
#ratio test
def filter_matches(matches,threshold):
    filter_matches=[]
    for x, y in matches:
        if x.distance <= y.distance*threshold:
            filter_matches.append(x)
    return filter_matches
    

In [10]:
def visualize_matches(img1,kp1,img2,kp2,matches, flags=2):
    match_img= cv2.drawMatches(img1,kp1,img2,kp2,matches,None,flags, matchColor= (100,255,100))
    plt.figure(figsize=(16,6),dpi=100)
    plt.imshow(match_img)

In [11]:
def estimate_motion(kp1,kp2,k,matches,depth, max_depth=3000):
    #compute the image in pixel  
    image_points1= [kp1[m.queryIdx].pt for m in matches]
    image_points2= [kp2[m.trainIdx].pt for m in matches]

    # extracting parameters from k
    fx=k[0][0]
    fy=k[1][1]
    cx=k[0][2]
    cy=k[1][2]

    #converting interesting image_points1 from 2d to 3d
    objects_list=np.zeros((0,3))
    delete=[]

    for i , (u,v) in enumerate(image_points1):
        z= depth[int(round(v))][int(round(u))]

        if z >= max_depth:
            delete.append(i)
            continue

        x= z*(u-cx)/fx
        y= z*(v-cy)/fy
        objects_list= np.vstack([objects_list, np.array([x,y,z])])

    image_points1= np.delete(image_points1, delete, 0)
    image_points2= np.delete(image_points2, delete, 0)

    _, rvec,tvec, inliers= cv2.solvePnPRansac(objects_list, image_points2, k, None)
    rvec = cv2.Rodrigues(rvec)[0]
    
    return rvec, tvec

    

In [26]:
def visual_odometry(handler, stereo_matcher="bm", detector= "orb", matcher="BF", filter_threshold=None, mask= True, 
                    verbose=False, plot=True):
    print(f"Computing disparityMap with stereo_matcher : {stereo_matcher}")
    print(f"Detecting features with detector {detector}")
    print(f"Matching features with {matcher} and filtered with ratio test at threshold : {filter_threshold}")

    if plot:
        fig = plt.figure(figsize=(14, 14))
        ax = fig.add_subplot(projection='3d')
        ax.view_init(elev=-20, azim=270)
        xs = handler.poses[:, 0, 3]
        ys = handler.poses[:, 1, 3]
        zs = handler.poses[:, 2, 3]
        ax.set_box_aspect((np.ptp(xs), np.ptp(ys), np.ptp(zs)))
        ax.plot(xs, ys, zs, c='k')        
        
    num_frames= handler.num_frames
    
    k_left,r_left,t_left = decomposeProjectionMatrix(handler.P0)

    trajectory=np.zeros((num_frames, 3,4))
    #print(trajectory)
    T_tot=np.eye(4)
    trajectory[0]=(T_tot[:3,:])

    #Compute the trajectory for every frame
    for i in range(num_frames-1):
        start=datetime.datetime.now()
        
        left_img=handler.left_images[i]
        right_img=handler.right_images[i]
        left_imgplus1=handler.left_images[i+1]

        depth= stereo_to_depth(left_img, right_img, handler.P0, handler.P1)
        #
        kp0,des0 = extract_features(left_img)
        kp1,des1 = extract_features(left_imgplus1)

        #okk

        unmatched_features= match_features(des0,des1)

        print(f"Numero features senza filtering: {len(unmatched_features)}")

        if filter_threshold is not None :
            matched_features=filter_matches(unmatched_features, filter_threshold)
        else:
            matched_features= unmatched_features #problem : list of 2-element list, ransac doesn't want it

        print(f"Numero features con filtering: {len(matched_features)}")
        #okk
        
        rvec,tvec= estimate_motion(kp0,kp1,k_left, matched_features, depth)
        
        #computing the transformation matrix [rvec| tvec]
        T_mat= np.hstack([rvec,tvec])

        #computing the homogeneous transformation matrix, so then I can invert it
        T_mat=np.vstack([T_mat, [0,0,0,1]])

        #inverting the transformation matrix
        T_mat=np.linalg.inv(T_mat)

        T_tot = T_tot.dot(T_mat)

        #appending T_tot into trajectory

        trajectory[i+1, :, :] = T_tot[:3][:]
        end=datetime.datetime.now()
        print(f"Frame {i+1} processato. Tempo trascorso: {end-start}")

        print(f"\n {trajectory[i+1,:,:].round(4)}\n\n\n")

        if plot:
            xs = trajectory[:i+2, 0, 3]
            ys = trajectory[:i+2, 1, 3]
            zs = trajectory[:i+2, 2, 3]
            plt.plot(xs, ys, zs, c='chartreuse')
            plt.pause(1e-32)

    if plot:
        plt.close()

    return trajectory
        
    

In [34]:
def calculate_error(ground_truth, estimated, error_type='mse'):
    '''
    Takes arrays of ground truth and estimated poses of shape Nx3x4, and computes error using
    Euclidean distance between true and estimated 3D coordinate at each position.
    
    Arguments:
    ground_truth -- Nx3x4 array of ground truth poses
    estimated -- Nx3x4 array of estimated poses
    
    Optional Arguments:
    error_type -- (str) can be 'mae', 'mse', 'rmse', or 'all' to return dictionary of all 3
    
    Returns:
    error -- either a float or dictionary of error types and float values
    
    '''
    # Find the number of frames in the estimated trajectory to compare with
    nframes_est = estimated.shape[0]
    
    def get_mse(ground_truth, estimated):
        se = np.sqrt((ground_truth[nframes_est, 0, 3] - estimated[:, 0, 3])**2 
                    + (ground_truth[nframes_est, 1, 3] - estimated[:, 1, 3])**2 
                    + (ground_truth[nframes_est, 2, 3] - estimated[:, 2, 3])**2)**2
        mse = se.mean()
        return mse
    
    def get_mae(ground_truth, estimated):
        ae = np.sqrt((ground_truth[nframes_est, 0, 3] - estimated[:, 0, 3])**2 
                    + (ground_truth[nframes_est, 1, 3] - estimated[:, 1, 3])**2 
                    + (ground_truth[nframes_est, 2, 3] - estimated[:, 2, 3])**2)
        mae = ae.mean()
        return mae
    
    if error_type == 'mae':
        return get_mae(ground_truth, estimated)
    elif error_type == 'mse':
        return get_mse(ground_truth, estimated)
    elif error_type == 'rmse':
        return np.sqrt(get_mse(ground_truth, estimated))
    elif error_type == 'all':
        mae = get_mae(ground_truth, estimated)
        mse = get_mse(ground_truth, estimated)
        rmse = np.sqrt(mse)
        return {'mae': mae,
                'rmse': rmse,
                'mse': mse}

In [27]:
def main():
    calib_path="KITTI_dataset/sequences/00/calib.txt"
    images_path="KITTI_dataset/sequences/00/"
    gtPoses_path="KITTI_dataset/poses/00.txt"
    handler = Dataset_Handler(calib_path,images_path,gtPoses_path)
    trajectory= visual_odometry(handler, filter_threshold=0.5)

In [36]:
if __name__ == "__main__":
    main()

Loading images, just a moment...: 100%|████████████████████████████████████████████| 4541/4541 [02:09<00:00, 34.99it/s]


Computing disparityMap with stereo_matcher : bm
Detecting features with detector orb
Matching features with BF and filtered with ratio test at threshold : 0.5
Tempo impiegato da bm:
0:00:00.011257
Numero features senza filtering: 500
Numero features con filtering: 106
Frame 1 processato. Tempo trascorso: 0:00:00.032046

 [[ 1.     -0.0026 -0.0028 -0.0261]
 [ 0.0026  1.     -0.0023 -0.005 ]
 [ 0.0028  0.0023  1.      0.6479]]



Tempo impiegato da bm:
0:00:00.010026
Numero features senza filtering: 500
Numero features con filtering: 122
Frame 2 processato. Tempo trascorso: 0:00:00.033612

 [[ 1.     -0.0022 -0.0043 -0.1192]
 [ 0.0022  1.     -0.0034 -0.018 ]
 [ 0.0043  0.0034  1.      1.2966]]



Tempo impiegato da bm:
0:00:00.008284
Numero features senza filtering: 500
Numero features con filtering: 118
Frame 3 processato. Tempo trascorso: 0:00:00.035415

 [[ 1.     -0.0025 -0.0078 -0.1393]
 [ 0.0025  1.     -0.0046 -0.0247]
 [ 0.0078  0.0046  1.      2.0079]]



Tempo impiegato da bm:

KeyboardInterrupt: 